In [3]:
from gurobipy import *
import numpy as np
import pandas as pd

# Structure


In [8]:
Biomass  = pd.read_csv("pred_2019.csv")["2018"].values

In [9]:
Biomass.sum()

355990.5464952136

In [4]:
DistanceB = pd.read_csv("Distance_Matrix.csv").drop(columns = ["INDEX"])
DistanceP = DistanceB.copy()

In [5]:
Di,Dj = DistanceB.shape 

In [6]:
M = 10**9
for i in range(Di):
    for j in range(Dj):
        if(i == j):
            DistanceB[i][j] = M
            DistanceP[i][j] = 0

In [12]:
Y = pd.read_csv("Depots.csv").values
Z = pd.read_csv("Refineries.csv").values

3.0

# Variables


In [13]:
m = Model()

In [14]:
B = m.addMVar((Di,Dj), vtype= GRB.CONTINUOUS , lb = 0 , name = "B" )

In [15]:
P = m.addMVar((Di,Dj), vtype= GRB.CONTINUOUS , lb = 0 , name= "P")

In [16]:
X = m.addMVar((Di), vtype= GRB.BINARY, name = "X")

In [17]:
Xm = m.addMVar((Di) , vtype= GRB.CONTINUOUS, name = "Xm")
Ym = m.addMVar((Di) , vtype= GRB.CONTINUOUS, name = "Ym")
Zm = m.addMVar((Di) , vtype= GRB.CONTINUOUS, name = "Zm")

In [ ]:
babs = m.addVar(vtype= GRB.CONTINUOUS , name = "babs" , lb = 0)

# Objective

In [20]:
import gurobipy as gp

In [21]:
exp1 = (DistanceB*B).sum()

In [22]:
exp1 += (DistanceP*P).sum()

In [23]:
exp2 = (20000*Y - B.sum(axis=0)).sum()

In [24]:
exp3 = (100000*Z - P.sum(axis=0)).sum()

In [25]:
m.setObjective(exp1 + exp2 + exp3 , GRB.MINIMIZE)

In [26]:
m.update()

# Constraints

In [27]:
x_mask = []
for i in range(Di):
    x_mask = B[i].tolist()
    m.addConstr(Xm[i] == max_(x_mask,constant = 0), name=f"C1m{i}")
    m.addConstr(X[i] == min_(Xm[i],constant = 1), name=f"C1{i}")

In [28]:
for i in range(Di):
    for j in range(Dj):
        m.addConstr(B[i][j] == Y[j] * B[i][j], name = f"C2_{i}_{j}")

In [29]:
for i in range(Di):
    for j in range(Dj):
        m.addConstr(P[i][j] == Z[j] * P[i][j], name = f"C3_{i}_{j}")

In [30]:
for i in range(Di):
    m.addConstr(B.sum(axis=1)[i] - Biomass[i] <= 0 , name=f"C_4_{i}")

In [31]:
for j in range(Dj):
    m.addConstr(B.sum(axis=0)[j] - 20000 <= 0 , name = f"C_5_{j}")

In [32]:
for j in range(Dj):
    m.addConstr(P.sum(axis=0)[j] - 100000 <= 0 , name = f"C_6_{j}")

In [37]:
con9 = (P.sum() >= (0.8*(Biomass.sum()+1)))

In [38]:
m.addConstr(con9 , name = "C_9")

<MConstr () *awaiting model update*>

In [40]:
for i in range(Di):
    m.addConstr(X[i] + Y[i] <= 1, name = f"C_11_{i}" )

In [41]:
for i in range(Di):
    m.addConstr(X[i] + Z[i] <= 1, name = f'C_12_{i}')

In [ ]:
m.addConstr(babs == (B.sum() - P.sum()) , name = "C_10_1")

In [ ]:
m.addConstr(babs <= 0.001, name = "C_10_2")

In [43]:
m.update()

In [44]:
m.write("2018.lp")

In [45]:
m.optimize()

Gurobi Optimizer version 10.0.2 build v10.0.2rc0 (win64)

CPU model: 12th Gen Intel(R) Core(TM) i5-12450H, instruction set [SSE2|AVX|AVX2]
Thread count: 8 physical cores, 12 logical processors, using up to 12 threads

Optimize a model with 20503 rows, 20402 columns and 88002 nonzeros
Model fingerprint: 0x266b732b
Model has 200 general constraints
Variable types: 20302 continuous, 100 integer (100 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 1e+09]
  Bounds range     [3e-03, 1e+00]
  RHS range        [1e+00, 4e+04]
  GenCon const rng [1e+00, 1e+00]
Presolve removed 17081 rows and 15540 columns
Presolve time: 0.19s
Presolved: 3422 rows, 4862 columns, 14339 nonzeros
Variable types: 3242 continuous, 1620 integer (1620 binary)
Found heuristic solution: objective 1.008845e+07
Found heuristic solution: objective 1.004170e+07

Root relaxation: objective 4.108390e+06, 1454 iterations, 0.01 seconds (0.01 work units)

    Nodes    |    Current Node 

In [46]:
obj = m.getObjective()

In [47]:
print(obj.getValue())

4108742.949999998


In [48]:
m.write("2018.sol")

# Extract results


In [49]:
def Sol_pipeline(m):
    
    # Biomass
    
    res = []
    for i in range(Di*Dj):
        res.append((m.getVars()[i].X))
    B_res = np.array(res)
    B_res = B_res.reshape(Di,Dj)
    idx = Di*Dj 
    
    # Pellets
    
    res.clear()
    for i in range(idx,(2*idx)):
        res.append((m.getVars()[i].X))
    P_res = np.array(res).reshape(Di,Dj)
    idx = 2*idx
    
    # Harvesting sites
    
    res.clear()
    for i in range(idx,(idx+Di)):
        res.append((m.getVars()[i].X))
    X_res = np.array(res)
    idx = idx + Di
    
    
    return {"B" : B_res , "P" : P_res , "X" : X_res}

In [50]:
result = Sol_pipeline(m)

In [52]:
B_res = result["B"]
P_res = result["P"]
X_res = result["X"]
Y_res = Y
Z_res = Z

In [54]:
P_sum = np.zeros((Dj))
B_sum = np.zeros((Di))
Depot_sum = np.zeros((Di))
Refinery_sum = np.zeros((Dj))

In [55]:
# Round off
for i in range(Di):
    for j in range(Dj):
        B_res[i][j] = round(B_res[i][j],2)
B_sum = B_res.sum(axis = 1)
P_sum = P_res.sum(axis = 1)
Depot_sum = B_res.sum(axis = 0)
Refinery_sum = P_res.sum(axis = 0)

In [56]:
B_res.sum()

43525.6

# Checks

In [57]:
# All values (forecasted biomass, biomass demand-supply, pellet demand-supply) must be
# greater than or equal to zero. 

print(False not in (Biomass >= 0),
False not in (B_res >= 0),
False not in (P_res >= 0 ))

True True True


In [58]:
# The amount of biomass procured for processing from each harvesting site ′𝑖′ must be less than
# or equal to that site’s forecasted biomass.

False not in (Biomass  - B_sum >= 0 )


True

In [59]:
# Total biomass reaching each preprocessing depot ′𝑗′ must be less than or equal to its yearly
# processing capacity (20,000).

False not in (Depot_sum <= 4000)

True

In [60]:
# Total pellets reaching each refinery ′𝑘′ must be less than or equal to its yearly processing
# capacity (100,000).

False not in (Refinery_sum <= 20000)

True

In [63]:
# Number of depots should be less than or equal to 25.

(Y_res.sum() <= 25)

True

In [64]:
# Number of refineries should be less than or equal to 5.

(Z_res.sum() <= 5)

True

In [65]:
# At least 80% of the total forecasted biomass must be processed by refineries each year.
(B_res.sum() >= (0.8 * Biomass.sum()))

True

In [66]:
# Total amount of biomass entering each preprocessing depot is equal to the total amount of
# pellets exiting that depot (within tolerance limit of 1e-03).

(B_res.sum() - P_res.sum()) <= 0.001 

True

# Objective estimate

In [68]:
residual1 = (Y_res.sum()*25000 - B_sum.sum())

In [69]:
residual2 = (Z_res.sum()*100000 - P_sum.sum())

In [70]:
transcost = (DistanceB*B_res).sum() + (DistanceP*P_res).sum()

In [71]:
estobj = residual1 + residual2 + transcost

In [74]:
estobj - obj.getValue()

2.3283064365386963e-09

# Final output

In [85]:
# DEPOTS
Dtest = []
for i in range(Di):
    if(Y_res[i] == 1):
        Dtest.append(i)
print(np.array(Dtest).min(), np.array(Dtest).max())
pd.DataFrame(Dtest, columns = ["Depot_loc"]).to_csv("18Depots",index = False)

14 95


In [86]:
# Refineries
Rtest = []
for i in range(Di):
    if(Z_res[i] == 1):
        Rtest.append(i)
print(np.array(Rtest).min(), np.array(Rtest).max())
pd.DataFrame(Dtest, columns = ["Refinery_loc"]).to_csv("18Refineries",index = False)

62 83


In [87]:
# Harvesting sites
Htest = []
for i in range(Di):
    if(X_res[i] == 1):
        Htest.append(i)
print(np.array(Htest).min(), np.array(Htest).max())
pd.DataFrame(Htest, columns = ["Harvesting_loc"]).to_csv("18HarvestSites",index = False)

0 99


In [118]:
# Biomass supply_demand
Btest = {"Source" : [], "Destination":[], "Value" : []}
for i in range(Di):
    for j in range(Dj):
        if(B_res[i][j] != 0):
            Btest["Source"].append(i)
            Btest["Destination"].append(j)
            Btest["Value"].append(B_res[i][j])
np.array(Btest["Value"]).sum() >= 0.8*Biomass.sum()
pd.DataFrame(Btest).to_csv("18Biomass_supply.csv",index=False)

In [122]:
# Pellets supply demand
Ptest = {"Source" : [], "Destination":[], "Value" : []}
for i in range(Di):
    for j in range(Dj):
        if(P_res[i][j] != 0):
            Ptest["Source"].append(i)
            Ptest["Destination"].append(j)
            Ptest["Value"].append(P_res[i][j])
np.array(Ptest["Value"]).sum() >= 0.8*Biomass.sum()
pd.DataFrame(Ptest).to_csv("18Pellets_supply.csv",index=False)